In [15]:
# !pip install langchain
# ! pip install pypdf

In [16]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [23]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("pdfs/Namami-Gange-Programme.pdf")
pages = loader.load()

In [24]:
len(pages), pages[0]

(9,
 Document(page_content='duties, AND NAMAMI GANGE PROGRAMME  \nIntroduction  \n Ganga is  considered as  the most sacred river in the  country with a unique \ncultural and spiritual significance.  It occupies  a specia l place in every Indian’s \nheart. Traversing over 2,500 km,  River Ganga is highly vene rated and millions of \npeople from its origin in the Gangotri g lacier in the Himalayas to the Sunderbans \ndelta in Bangladesh  depend on its water for their  livelihood . \nRiver Network  \n The River Ganga originates in the Himalayas with several major head -\nstreams – Alaknanda, Bhagirathi, Bhilangana, Dhauliganga, Mandakini,  Nandakini \nand Pindar, which progressively join together on or before Devaprayag. \nDescending in the plains, the river flows approximately southeast and is joined by \nseveral large streams such as Ramganga, Yamuna, Kosi, Gandak, Gomti, Sone, \nKaramnasa and Gh aghra to become an immense river in the plains below \nAllahabad. The river then flows thr

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [53]:
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [57]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", "(?<=\. )"]
)

In [58]:
docs = text_splitter.split_documents(pages)

In [59]:
len(docs), len(pages)

(27, 9)

In [60]:
docs[0]

Document(page_content='duties, AND NAMAMI GANGE PROGRAMME  \nIntroduction  \n Ganga is  considered as  the most sacred river in the  country with a unique \ncultural and spiritual significance.  It occupies  a specia l place in every Indian’s \nheart. Traversing over 2,500 km,  River Ganga is highly vene rated and millions of \npeople from its origin in the Gangotri g lacier in the Himalayas to the Sunderbans \ndelta in Bangladesh  depend on its water for their  livelihood . \nRiver Network  \n The River Ganga originates in the Himalayas with several major head -\nstreams – Alaknanda, Bhagirathi, Bhilangana, Dhauliganga, Mandakini,  Nandakini \nand Pindar, which progressively join together on or before Devaprayag. \nDescending in the plains, the river flows approximately southeast and is joined by \nseveral large streams such as Ramganga, Yamuna, Kosi, Gandak, Gomti, Sone, \nKaramnasa and Gh aghra to become an immense river in the plains below', metadata={'source': 'pdfs/Namami-Gange-P

In [56]:
docs[0]

Document(page_content='duties, AND NAMAMI GANGE PROGRAMME  \nIntroduction  \n Ganga is  considered as  the most sacred river in the  country with a unique \ncultural and spiritual significance.  It occupies  a specia l place in every Indian’s \nheart. Traversing over 2,500 km,  River Ganga is highly vene rated and millions of \npeople from its origin in the Gangotri g lacier in the Himalayas to the Sunderbans \ndelta in Bangladesh  depend on its water for their  livelihood . \nRiver Network  \n The River Ganga originates in the Himalayas with several major head -\nstreams – Alaknanda, Bhagirathi, Bhilangana, Dhauliganga, Mandakini,  Nandakini \nand Pindar, which progressively join together on or before Devaprayag. \nDescending in the plains, the river flows approximately southeast and is joined by \nseveral large streams such as Ramganga, Yamuna, Kosi, Gandak, Gomti, Sone, \nKaramnasa and Gh aghra to become an immense river in the plains below', metadata={'source': 'pdfs/Namami-Gange-P